# Balance Inverted Pendulum

### Problem Formulation

MDP: <br>
Let state X be pair of $(x_1,x_2)$ which is (angle, velocity). The angle is $\in [0,2\pi]$. The velocity is $\in [-Vmax, Vmax]$. By discretizing the state, we get state X as N1xN2 matrix, where N1 is the density of angle, and N2 is the density of the velocity. (N1 = 100, N2 = 50, Vmax = 3 by default) <br>
Let control U $\in [-Umax, Umax]$. By discretizing the U, we get U as a N3-length array, where N3 is the density of the control. (Umax = 1, N3 = 30, by default) <br>
Let stage cost be $l(x,u)dt = (1-exp(kcosx_1-k)+\frac{r}{2}u^2)dt$ <br>
Let motion model $p_f(x'|x,u) \sim N(x+f(x,u)dt,\sigma\sigma^Tdt)$, where $f(x,u) = [x_2,a*sinx_1-b*x_2+u]$. (a=1, b=0.8, dt=1, k=100, r=1, $\sigma$ = 0.1 by default) <br>

Optimization problem: <br>
The value function V is a N1xN2 matrix where each entry cooresponding to optimal value of each state. <br>
The policy P is a N1xN2 matrix where each entry cooresponding to optimal control of each state. <br>
Also, for the discount factor $\gamma$ is set equal to 0.9 as default. <br>
<br>
Interpolation problem: <br>
Let t be an array that stores discrete time.
Let x1 be an array that stores the x-axis positions over the time.
Let x2 be an array that stores the y-axis positions over the time.
Let theta be an array that stores angles over the time.
Let u be an array that stores the controls over the time.

### Technical Approach

Initalize motion model $p_f$, which is a matrix with dimensionN1xN2xN3xN1xN2: <br>
For each state-action pair(x1,x2,u): 1. Calculate out $x+f(x,u)dt$, which is the mean of $p_f(x'|x1,x2,u)$. 2. for each state, calculate out its distance to the mu. Notice that the angle 0 to 2*pi is circular, thus the distance between two angles should be $min(|\theta1-\theta2|, 2\pi-|\theta1-\theta2|)$. 3. Assign guassian probability ($\sigma\sigma^Tdt$ covariance) to each states according to how far they are actually away from the mean. 4. Normalize the probabilies of all states so that they sum up to one. <br>
<br>
Value Iteration: <br>
Intialize $p_f$ (Set $p_f(x'|x,u) \sim N(x+f(x,u)\delta t, \sigma \sigma^T \delta t)$) <br>
Initialize V for each state <br>
loop until convergence: <br>
&emsp; for all x in X:<br>
&emsp;&emsp; $V^{t+1}(x) = min_u(l(x,u)\delta t+\gamma \sum_{x'\in X}[p_f(x'|x,u)*V^t(x')])$<br>
<br>
Policy Iteration: <br>
Intialize $p_f$ (Set $p_f(x'|x,u) \sim N(x+f(x,u)\delta t, \sigma \sigma^T \delta t$) <br>
Initialize Policy for each state <br>
loop until convergence: <br>
&emsp; for all x in X: <br>
&emsp; &emsp; u <-- Current Policy, $P^t$ <br>
&emsp;&emsp; Get linear equation: $V(x) = l(x,u)\delta t+\gamma \sum_{x'\in X}[p_f(x'|x,u)*V(x')])$<br>
&emsp; Solve the above linear equations as a big linear system (Use python np.linalg.solve(a,b))<br>
&emsp; Then you get V for all states <br>
&emsp; for all x in X: <br>
&emsp;&emsp; $P^{t+1}(x) = argmin_u(l(x,u)\delta t+\gamma \sum_{x'\in X}[p_f(x'|x,u)*V(x')])$ <br>
<br>
Interpolation: <br>
Following the code in PendulumProblem.py, all we need to do is: <br>
Intialize theta_i, and v_i<br>
Let $self.theta = [], self.u = []$ <br>
for i in range(self.t):<br>
&emsp; Get u from policy by mapping theta_i,v_i to closet pair in discretized states <br>
&emsp; Update theta_i, v_i by randomly generate (angle, velocity) pair from motion model $p_f$ <br>
&emsp; self.theta.append(theta_i), self.u.append(u) <br>
Then, self.x1 = np.sin(self.theta), self.y1 = np.cos(self.theta). <br>
Then, just run the code in PendulumProblem.py. <br>

### Result

The hyperparameters I used are the dafult value I set in problem formulation. Overall, It is hard and takes me many times to find a good set of hyperparameters. One thing interesting is that higher the covariance sigma is, more blurry the optimal value function is. Lower the variance/noise, the optimal policy will be more smooth. If the sigma/noise is larger than 0.2, the pendulum will not achieve stable equilibrium in the vertical state, since the noise is too high which makes the model hard to predict the next state, thus, hard to achieve equilibrium.
(One more reason why noise makes pendulum to be stable: the noise is also applied to the model to the interpolation testing which is animated. Thus, in the animation, the pendulum will oscillate with small magnitude around the 0 angle. Higher the sigma is, higher the intensity of the oscillation. ) <br>
In terms of runtime, PI is much faster than VI. PI only takes 8 iterations to converge while VI takes 101. <br>
The runtime, performance and testing all worked as expected with a good set hyperparameter, since VI and PI both guarantee the optimal policy, thus with same hyperparameters, the optimal value function and policy of VI and PI are the same.<br>